<a href="https://colab.research.google.com/github/Seher-Kanwal/Sentimental-analysis-Using-bert-base-multilingual-uncased-sentiment/blob/main/SentimentalAnalysisOnYelpReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1 - First we will import all the dependcies**


* First we import transformer - a libray of hugging face in order to import    pre-trainted models

*  Import tokenizer than convert text into some vector representation and than we import AutoModelSequence to load our model for sentimental analysis

*   Request - bcz we are going to send a req to yelp to scarp comments 

*   Beautiful - to scrap comments

*   pandas - to deal with dataframes

*   numpy - to perfrom mathematical operations

*   torch - to use argmax fn to get high priorty sentiment









In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 15.3 MB/s 
     |████████████████████████████████| 7.6 MB 77.4 MB/s 
     |████████████████████████████████| 182 kB 49.3 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [5]:
!pip install requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
! pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import torch
import requests
from bs4 import BeautifulSoup
import re

# **Instantiate Model**
Here We load tokenizer according to our sentimental model we are going to use and also load the model for sentimental analysis



In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/669M [00:00<?, ?B/s]

 
 
 # **Encode and Calculate Sentiment**

 - here we use return_tensor bcz our model worked on tensor (pytorch tensors)

In [26]:
token = tokenizer.encode("I hate this, absolutely the worst", return_tensors='pt')

In [27]:
token

tensor([[  101,   151, 39487, 10372,   117, 35925, 10563, 10103, 43060,   102]])

In [28]:
result = model(token)

In [32]:
### This model return the result in the logistic tag for the sentimental scores 
tokenizer.decode(token[0][1])

'i'

In [33]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 5.0607,  1.6029, -1.0123, -3.0154, -1.8469]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [35]:
result.logits

tensor([[ 5.0607,  1.6029, -1.0123, -3.0154, -1.8469]],
       grad_fn=<AddmmBackward0>)

## **As model gives rattings between 1 and 5**
but when we axtract using pytorch it starts from 0 to 4

in order to convert it into 1 to 5 we add 1 in all the ratings after extracting the argmax

In [37]:
int (torch.argmax(result.logits))+1

1

# **Now Will Scrap the comments from the yelp**

In [38]:
r = requests.get('https://www.yelp.com/biz/brendas-french-soul-food-san-francisco-6')

In [39]:
## 200 means Request was made successfully
r

<Response [200]>

In [40]:
r.text

'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b05852393ae5/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>\n            window.yelp = window.yelp || {};\

### **Now we will be parsing html data we got**

In [43]:
soup = BeautifulSoup(r.text, 'html.parser')

## The re.compile() method
re.compile(pattern, repl, string):

 We can combine a regular expression pattern into pattern objects, which can be used for pattern matching. It also helps to search a pattern again without rewriting it.

 https://www.tutorialspoint.com/Why-do-we-use-re-compile-method-in-Python-regular-expression

In [56]:
regres = re.compile('.*comment.*')

In [57]:
results = soup.findAll('p',{'class': regres})

In [58]:
reviews = [result.text for result in results]

In [60]:
reviews[0]

"Today I go there with my coworker and then have orders foods After having the food We settle bills and the server charges more than usual price After that I checked and explained to him He not \xa0listen my explainingHe very rude on me and The food is ok Please take care yours business reputation He is not skilful person on service Try to be good employeesI'm soo sad for your business"

# 5. **Load Reviews into DataFrame and Score**

In [61]:
import numpy as np
import pandas as pd

In [62]:
df = pd.DataFrame(np.array(reviews), columns = ['Comment'])

In [63]:
df.head()

,Comment
0,Today I go there with my coworker and then hav...
1,"Tommy, thank you so much for taking the time t..."
2,¿Busca un grifo de cocina elegante y con estil...
3,The soul food is in my opinion starting to get...
4,"Hi Day, it’s a shame that you didn’t enjoy you..."


In [64]:
df.shape

(20, 1)

## **Here we will write a fn that takes the comments or reviews and return the ratings just like the above that we did for a single statement to extract Sentimental scores**

In [65]:
def sentimental_rattings(review):
  token = tokenizer.encode(review, return_tensors= 'pt')
  res = model(token)
  return int(torch.argmax(res.logits))+1


In [72]:
sentimental_rattings(df['Comment'].iloc[1])

5

## The max len of a string that we can pass to that model is 512 so we only passing the first 512 characters

In [74]:
df['sentiment_score'] = df['Comment'].apply(lambda x: sentimental_rattings(x[:512]))

In [75]:
df.head()

,Comment,sentiment_score
0,Today I go there with my coworker and then hav...,1
1,"Tommy, thank you so much for taking the time t...",5
2,¿Busca un grifo de cocina elegante y con estil...,5
3,The soul food is in my opinion starting to get...,3
4,"Hi Day, it’s a shame that you didn’t enjoy you...",2


In [76]:
df['Comment'][0]

"Today I go there with my coworker and then have orders foods After having the food We settle bills and the server charges more than usual price After that I checked and explained to him He not \xa0listen my explainingHe very rude on me and The food is ok Please take care yours business reputation He is not skilful person on service Try to be good employeesI'm soo sad for your business"

In [78]:
df['sentiment_score'][0]

1